## Dependencies:

In [1]:
import pandas as pd
import numpy as np

from src.data.utils import read_data

## Configuration

In [2]:
pd.set_option('max_colwidth', 500)
pd.set_option('display.max_rows', 500)

## Get data

In [3]:
read_data('../data/raw/FoodData_Central_csv_2023-04-20/all_downloaded_table_record_counts.csv')

,Table,Number of Records
0,acquisition_samples,7187
1,agricultural_samples,811
2,branded_food,1807155
3,fndds_derivation,50
4,fndds_ingredient_nutrient_value,273911
5,food,1875578
6,food_attribute,2100036
7,food_attribute_type,6
8,food_calorie_conversion_factor,4973
9,food_category,29


## Read data

In [4]:
data_path = '../data/raw/FoodData_Central_csv_2023-04-20/'

food = read_data(f'{data_path}food.csv')
survey_fndds_food = read_data(f'{data_path}survey_fndds_food.csv')
nutrient_data = read_data(f'{data_path}food_nutrient.csv')
nutrient_info = read_data(f'{data_path}nutrient.csv')
input_food = read_data(f'{data_path}input_food.csv')
fndds_ingredient_nutrient_value = read_data(f'{data_path}fndds_ingredient_nutrient_value.csv')
fndds_derivation = read_data(f'{data_path}fndds_derivation.csv')
#category = pd.read_csv(f'{data_path}food_category.csv')
component = read_data(f'{data_path}food_component.csv')
category = read_data(f'{data_path}wweia_food_category.csv')

/mnt/c/Users/dhdzm/Documents/projects/ingeineria_en_alimentos/ingenieria_en_alimentos/ai_in_food/src/data/utils.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [5]:
nutrient_info.loc[
    nutrient_info.nutrient_nbr.notna(), 'nutrient_nbr_int'
] = nutrient_info.loc[nutrient_info.nutrient_nbr.notna()].nutrient_nbr.astype('int')

fndss_dt = food.data_type == 'survey_fndds_food'
relevant_list = ['description', 'fdc_id', 'food_category_id', 'publication_date']
food_survey_food = survey_fndds_food.merge(food[fndss_dt][relevant_list], on='fdc_id', how='left')

food_survey_food = food_survey_food\
.merge(category, how='left', left_on='wweia_category_code', right_on='wweia_food_category')\
.drop(['wweia_category_code', 'food_category_id', 'start_date', 'end_date', 'publication_date'], axis=1)

In [6]:
def get_food_information(test_food):
    
    food_survey_food[food_survey_food.fdc_id == test_food]

    ingredients = input_food[input_food.fdc_id == test_food].sort_values('seq_num')
    codes = ingredients.sr_code.unique()

    nutrient_info_reduced = nutrient_info[nutrient_info.nutrient_nbr_int.notna()][['name', 'unit_name', 'nutrient_nbr_int']]
    specific_nutrient_val = fndds_ingredient_nutrient_value\
    [fndds_ingredient_nutrient_value['ingredient code'].isin(codes)]\
    .drop_duplicates(['ingredient code', 'Nutrient code'])\
    .merge(
        nutrient_info_reduced,
        left_on='Nutrient code',
        right_on='nutrient_nbr_int')\
    [['ingredient code', 'Nutrient code', 'Nutrient value', 'name', 'unit_name']]
    specific_nutrient_val = specific_nutrient_val[specific_nutrient_val['Nutrient value']!=0]\
    .sort_values(['Nutrient code', 'Nutrient value']).reset_index(drop=True)\
    .drop_duplicates(['ingredient code', 'Nutrient code'])

    missing_codes = list(set(codes) - set(specific_nutrient_val['ingredient code'].unique()))

    missing_ingredients = ingredients[ingredients.sr_code.isin(missing_codes)]


    nutrient_info_reduced = nutrient_info[nutrient_info.nutrient_nbr_int.notna()][['name', 'unit_name', 'nutrient_nbr_int']]
    specific_nutrient_val = fndds_ingredient_nutrient_value\
    [fndds_ingredient_nutrient_value['ingredient code'].isin(codes)]\
    .drop_duplicates(['ingredient code', 'Nutrient code'])\
    .merge(
        nutrient_info_reduced,
        left_on='Nutrient code',
        right_on='nutrient_nbr_int')\
    [['ingredient code', 'Nutrient code', 'Nutrient value', 'name', 'unit_name']]
    specific_nutrient_val = specific_nutrient_val[specific_nutrient_val['Nutrient value']!=0]\
    .sort_values(['ingredient code', 'Nutrient code', 'Nutrient value'])\
    .reset_index(drop=True)\
    .drop_duplicates(['ingredient code', 'Nutrient code'])
    return ingredients, missing_ingredients, specific_nutrient_val

In [7]:
food_survey_food_resume = food_survey_food.groupby('wweia_food_category').agg(
    count_vals=('food_code',  'count'),
    categ=('wweia_food_category_description',  'first'),
    description_text=('description', lambda x: x.to_list()),
    fdc_ids = ('fdc_id', lambda x: x.to_list())
)

In [8]:
food_survey_food_resume

,count_vals,categ,description_text,fdc_ids
wweia_food_category,,,,
1002,7,"Milk, whole","[Milk, whole, Milk, lactose free, whole, Buttermilk, whole, Goat's milk, whole, Milk, dry, reconstituted, whole, Milk, evaporated, NS as to fat content, Milk, evaporated, whole]","[2340762, 2340769, 2340771, 2340773, 2340775, 2340778, 2340779]"
1004,4,"Milk, reduced fat","[Milk, NFS, Milk, reduced fat (2%), Milk, lactose free, reduced fat (2%), Milk, evaporated, reduced fat (2%)]","[2340761, 2340763, 2340768, 2340780]"
1006,4,"Milk, lowfat","[Milk, low fat (1%), Milk, lactose free, low fat (1%), Kefir, NS as to fat content, Milk, dry, reconstituted, low fat (1%)]","[2340764, 2340766, 2340772, 2340776]"
1008,6,"Milk, nonfat","[Milk, fat free (skim), Milk, lactose free, fat free (skim), Buttermilk, fat free (skim), Milk, dry, reconstituted, NS as to fat content, Milk, dry, reconstituted, fat free (skim), Milk, evaporated, fat free (skim)]","[2340765, 2340767, 2340770, 2340774, 2340777, 2340781]"
1202,10,"Flavored milk, whole","[Milk, condensed, sweetened, Chocolate milk, ready to drink, whole, Chocolate milk, made from dry mix with whole milk, Chocolate milk, made from dry mix with whole milk (Nesquik), Chocolate milk, made from syrup with whole milk, Chocolate milk, made from light syrup with whole milk, Chocolate milk, made from sugar free syrup with whole milk, Hot chocolate / Cocoa, made with dry mix and whole milk, Hot chocolate / Cocoa, made with no sugar added dry mix and whole milk, Strawberry milk, whole]","[2340782, 2340847, 2340858, 2340863, 2340871, 2340876, 2340880, 2340882, 2340888, 2340894]"
1204,15,"Flavored milk, reduced fat","[Chocolate milk, ready to drink, reduced fat, Hot chocolate / Cocoa, ready to drink, Hot chocolate / Cocoa, ready to drink, with whipped cream, Chocolate milk, made from dry mix with reduced fat milk, Chocolate milk, made from dry mix, NS as to type of milk (Nesquik), Chocolate milk, made from dry mix with reduced fat milk (Nesquik), Chocolate milk, made from no sugar added dry mix with reduced fat milk (Nesquik), Chocolate milk, made from syrup, NS as to type of milk, Chocolate milk, made f...","[2340848, 2340854, 2340857, 2340859, 2340862, 2340864, 2340868, 2340870, 2340872, 2340877, 2340883, 2340889, 2340893, 2340895, 2340900]"
1206,11,"Flavored milk, lowfat","[Chocolate milk, ready to drink, low fat, Chocolate milk, ready to drink, reduced sugar, NS as to milk, Chocolate milk, ready to drink, low fat (Nesquik), Chocolate milk, ready to drink, low fat, no sugar added (Nesquik), Chocolate milk, made from dry mix with low fat milk (Nesquik), Chocolate milk, made from syrup with low fat milk, Chocolate milk, made from light syrup with low fat milk, Hot chocolate / Cocoa, made with dry mix and low fat milk, Hot chocolate / Cocoa, made with no sugar ad...","[2340850, 2340851, 2340852, 2340853, 2340865, 2340873, 2340878, 2340884, 2340890, 2340896, 2340898]"
1208,12,"Flavored milk, nonfat","[Chocolate milk, NFS, Chocolate milk, ready to drink, fat free, Hot chocolate / Cocoa, ready to drink, made with nonfat milk, Chocolate milk, made from dry mix with fat free milk, Chocolate milk, made from dry mix with fat free milk (Nesquik), Chocolate milk, made from syrup with fat free milk, Chocolate milk, made from light syrup with fat free milk, Hot chocolate / Cocoa, made with dry mix and water, Hot chocolate / Cocoa, made with dry mix and fat free milk, Hot chocolate / Cocoa, made wi...","[2340846, 2340849, 2340855, 2340860, 2340866, 2340874, 2340879, 2340881, 2340885, 2340887, 2340891, 2340897]"
1402,13,Milk shakes and other dairy drinks,"[Eggnog, Milk shake, home recipe, chocolate, Milk shake, home recipe, flavors other than chocolate, Milk shake, home recipe, chocolate, light, Milk shake, home recipe, flavors other than chocolate, light, Milk shake with malt, Milk shake, fast food, chocolate, Milk shake, fast food, flavors other than chocolate, Milk shake, bottled, chocolate, Milk shake, bottled, flavors othe

### We can select specific food categories and see which products are there.

In [11]:
milk = 1002, 1004, 1006, 1008, 1404, 9602, #9404, 9402,
cheese = 1602, 1604
yogurt = 1820, 1822, 9010, 
meat = 2002, 2004, 2006, 2008, 2010, 2602, 2604, 9007
birds = 2202, 2206
fish = 2402
rice = 4002
pasta = 4004
bread = 4202
juice = 7002, 7004, 7006, 7204, 9202
tea = 7304
beer = 7502
wine = 7504
water = 7702, 7704, 9204
cream = 8008
mayonnaise = 8010
sugar = 8802, 8804

### Now we can select the desired products in each category, and by manual labor delete the ones that might not work. 

In [12]:
food_dataframe = {}

In [13]:
not_categ = 2340899, 2340886, 2340875, 2340869, 2340867, 2340861, 2340856, 2340787, 2340788, 2340790, 2340792, 2340892
isin_criteria = food_survey_food.wweia_food_category.isin(milk)
food_dataframe['milk'] = food_survey_food[isin_criteria].set_index('fdc_id').drop(list(not_categ)).drop('food_code', axis=1)

not_categ = 2341154, 2341155, 2341160, 2341182, 2341183, 2341183, 2341221
isin_criteria = food_survey_food.wweia_food_category.isin(cheese)
food_dataframe['cheese'] = food_survey_food[isin_criteria].set_index('fdc_id').drop(list(not_categ)).drop('food_code', axis=1)

not_categ = None
isin_criteria = food_survey_food.wweia_food_category.isin(yogurt)
food_dataframe['yogurt'] = food_survey_food[isin_criteria].set_index('fdc_id').drop('food_code', axis=1)

In [14]:
relevant_indexes = []
for element in food_dataframe.values():
    relevant_indexes += element.index.to_list()

### Here we get the ingredients and nutrition information of the ingredients. 

In [21]:
food_ingredient_nutrition_info = {}
for food in relevant_indexes:
    food_ingredient_nutrition_info[food] = {}
    ingredients, missing_ingredients, nutrition = get_food_information(food)
    ingredients.index = ingredients.index.map(str)
    missing_ingredients.index = missing_ingredients.index.map(str)
    nutrition.index = nutrition.index.map(str)
    food_ingredient_nutrition_info[food]['ingredients'] = ingredients[['sr_code', 'sr_description']].to_dict()
    food_ingredient_nutrition_info[food]['missing_ingredients'] = missing_ingredients.sr_code.to_dict()
    food_ingredient_nutrition_info[food]['nutrition'] = nutrition.to_dict()
    food_ingredient_nutrition_info[food]['unique_ingredients_code'] = ingredients.sr_code.unique()
    food_ingredient_nutrition_info[food]['unique_ingredients'] = ingredients.sr_description.unique()

In [23]:
food_ingredient_nutrition_info_processed = pd.DataFrame(food_ingredient_nutrition_info).T
food_ingredient_nutrition_info_processed.to_parquet('../data/interim/food_ingredient_nutrition_info_processed.parquet')

### Similarity between specific products (if they belong to the same category, then it's a 1, if not, it's a 0)

In [196]:
response_dataframes = []
for key in food_dataframe.keys():
    for key2 in food_dataframe.keys():
        response=0
        if key==key2:
            response=1
        response_dataframe = food_dataframe[key].reset_index()[['fdc_id']]
        response_dataframe['fdc_id_to'] = [response_dataframe.fdc_id.to_list()]*len(response_dataframe)
        response_dataframe = response_dataframe.explode('fdc_id_to').reset_index(drop=True)
        response_dataframe['similar'] = response
        response_dataframes.append(response_dataframe)

In [197]:
response_dataframe = pd.concat(response_dataframes)

In [203]:
response_dataframe.to_csv('../data/interim/response_dataframe')